# Exploratory Data Analysis
- This file contains code and descriptions of exploratory data analysis

In [1]:
import pandas as pd
import numpy as np

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.io as pio
from plotly.subplots import make_subplots
import plotly.graph_objects as go

pio.templates.default = "plotly_dark"

In [2]:
fangraphs_data = pd.read_csv('../data/fangraphs_player_data.csv')
fangraphs_data

,IDfg,Season,Name,Team,Age,G,AB,PA,H,1B,...,maxEV,HardHit,HardHit%,Events,CStr%,CSW%,xBA,xSLG,xwOBA,L-WAR
0,15640,2024,Aaron Judge,NYY,32,158,559,704,180,85,...,117.5,238,0.609,391,0.146,0.267,NaN,NaN,NaN,11.3
1,15640,2022,Aaron Judge,NYY,30,157,570,696,177,87,...,118.4,246,0.609,404,0.169,0.287,NaN,NaN,NaN,11.4
2,25764,2024,Bobby Witt Jr.,KCR,24,161,636,709,211,123,...,116.9,259,0.481,538,0.138,0.236,NaN,NaN,NaN,10.0
3,13611,2018,Mookie Betts,BOS,25,136,520,614,180,96,...,110.6,217,0.500,434,0.220,0.270,NaN,NaN,NaN,10.4
4,10155,2018,Mike Trout,LAA,26,140,471,608,147,80,...,118.0,162,0.460,352,0.201,0.261,NaN,NaN,NaN,9.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1228,10815,2023,Jurickson Profar,- - -,30,125,459,521,111,73,...,108.8,119,0.317,375,0.151,0.236,NaN,NaN,NaN,-1.8
1229,6876,2017,Mark Trumbo,BAL,31,146,559,603,131,86,...,118.5,154,0.375,411,0.154,0.291,NaN,NaN,NaN,-1.8
1230,393,2018,Victor Martinez,DET,39,133,467,508,117,87,...,107.6,129,0.304,425,0.148,0.212,NaN,NaN,NaN,-1.7
1231,1177,2017,Albert Pujols,LAA,37,149,593,636,143,103,...,112.2,197,0.391,504,0.180,0.268,NaN,NaN,NaN,-1.9


- Since our target variable, wRC+ is a rate statistic (a statistic that rates a player's performance on aggregate), a player with a low number of plate appearances could have an extremely high or low wRC+ that does not actually reflect their true performance but rather is a result of a small number of plate appearances.
- The data loaded in should already have this in account, as the `baseball_data_loader` only scrapes players with more than 502 plate appearances. 500 plate appearances was chosen because is is the number of plate appearances required to be eligable for offensive awards. 


### Fangraphs Dataset


In [3]:
fangraphs_data.isnull().sum().index

Index(['IDfg', 'Season', 'Name', 'Team', 'Age', 'G', 'AB', 'PA', 'H', '1B',
       ...
       'maxEV', 'HardHit', 'HardHit%', 'Events', 'CStr%', 'CSW%', 'xBA',
       'xSLG', 'xwOBA', 'L-WAR'],
      dtype='object', length=320)

In [4]:
null_sums = fangraphs_data.isnull().sum()
columns_with_nans_count = null_sums[null_sums > 0]
columns_with_nans_count.keys()

Index(['Fld', 'phLI', 'KN%', 'KNv', 'XX%', 'PO%', 'wKN', 'wKN/C', 'FT% (sc)',
       'FO% (sc)', 'EP% (sc)', 'SC% (sc)', 'KN% (sc)', 'UN% (sc)', 'vFT (sc)',
       'vFO (sc)', 'vEP (sc)', 'vSC (sc)', 'vKN (sc)', 'FT-X (sc)',
       'FO-X (sc)', 'EP-X (sc)', 'SC-X (sc)', 'KN-X (sc)', 'FT-Z (sc)',
       'FO-Z (sc)', 'EP-Z (sc)', 'SC-Z (sc)', 'KN-Z (sc)', 'wFT (sc)',
       'wFO (sc)', 'wEP (sc)', 'wSC (sc)', 'wKN (sc)', 'wFT/C (sc)',
       'wFO/C (sc)', 'wEP/C (sc)', 'wSC/C (sc)', 'wKN/C (sc)', 'UBR', 'wGDP',
       'CS% (pi)', 'KN% (pi)', 'SB% (pi)', 'XX% (pi)', 'vCS (pi)', 'vKN (pi)',
       'vSB (pi)', 'vXX (pi)', 'CS-X (pi)', 'KN-X (pi)', 'SB-X (pi)',
       'XX-X (pi)', 'CS-Z (pi)', 'KN-Z (pi)', 'SB-Z (pi)', 'XX-Z (pi)',
       'wCS (pi)', 'wKN (pi)', 'wSB (pi)', 'wXX (pi)', 'wCS/C (pi)',
       'wKN/C (pi)', 'wSB/C (pi)', 'wXX/C (pi)', 'FRM', 'xBA', 'xSLG',
       'xwOBA'],
      dtype='object')

In [5]:
#dropping columns where at least 80% of the values are nan
fangraphs_data = fangraphs_data.drop((columns_with_nans_count.apply(lambda row: row/len(fangraphs_data)) > 0.8).index, axis = 1)

#filling other columns with the median value of the column
fangraphs_data = fangraphs_data.fillna(fangraphs_data.median)
fangraphs_data

,IDfg,Season,Name,Team,Age,G,AB,PA,H,1B,...,LA,Barrels,Barrel%,maxEV,HardHit,HardHit%,Events,CStr%,CSW%,L-WAR
0,15640,2024,Aaron Judge,NYY,32,158,559,704,180,85,...,18.9,105,0.269,117.5,238,0.609,391,0.146,0.267,11.3
1,15640,2022,Aaron Judge,NYY,30,157,570,696,177,87,...,14.9,106,0.262,118.4,246,0.609,404,0.169,0.287,11.4
2,25764,2024,Bobby Witt Jr.,KCR,24,161,636,709,211,123,...,15.1,77,0.143,116.9,259,0.481,538,0.138,0.236,10.0
3,13611,2018,Mookie Betts,BOS,25,136,520,614,180,96,...,18.5,57,0.131,110.6,217,0.500,434,0.220,0.270,10.4
4,10155,2018,Mike Trout,LAA,26,140,471,608,147,80,...,18.6,54,0.153,118.0,162,0.460,352,0.201,0.261,9.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1228,10815,2023,Jurickson Profar,- - -,30,125,459,521,111,73,...,14.8,15,0.040,108.8,119,0.317,375,0.151,0.236,-1.8
1229,6876,2017,Mark Trumbo,BAL,31,146,559,603,131,86,...,12.5,26,0.063,118.5,154,0.375,411,0.154,0.291,-1.8
1230,393,2018,Victor Martinez,DET,39,133,467,508,117,87,...,14.1,18,0.042,107.6,129,0.304,425,0.148,0.212,-1.7
1231,1177,2017,Albert Pujols,LAA,37,149,593,636,143,103,...,13.5,24,0.048,112.2,197,0.391,504,0.180,0.268,-1.9


In [6]:
sorted(fangraphs_data['Season'].unique())

[2015, 2016, 2017, 2018, 2019, 2021, 2022, 2023, 2024]

The features that were chosen for analysis are:
- EV: average exit veloxity 
- Hard Hit %: percentage of batted balls w/ exit velocity >95mph
- BB%: Walks/Plate Appearance
- K%: Strike out/Plate Appearance
- ISO: SLG - AVG, isolated power
- O-Swing%: Percentage of pitches outside of the zone a player swings at
- Z-Swing%: Percentage of pitches inside of the zone a player swings at
- O-Contact%: Percentage of times a batter makes contact with the ball when swinging at pitches outside the zone
- Z-Contact%: Percentage of times a batter makes contact with the ball when swinging at pitches inside the zone
- SwStr%: Percentage of pitches thrown that result in a batter swinging and missing
- CStr%: Percentage of pitches that are called strikes without the batter swinging 
- maxEV: Highest exit velocity by the batter recorded in a season
- Barrel%: Percent of batted balls that are counted as barrels 
- FB%: Percentage of balls put in play that result in a fly ball
- GB%: Percentage of balls put in play that result in a ground ball
- AVG: Batting average
- wRC+: Weighted Runs Created adjusted for league and ballpark factors
- LA: average angle that a ball flies off the player's bat

In a future project, I would like to use methods such as PCA for feature selection, but for this project, I will mainly be looking at these select features.

In [7]:
fg_select_data = fangraphs_data[['Name', 'Season', 'AVG','ISO', 'wRC+','BB%', 'K%', \
                'Barrel%', 'HardHit%', 'EV', 'maxEV', 'LA', 'GB%', 'FB%','SwStr%', 'CStr%', \
                'O-Swing%', 'Z-Swing%', 'O-Contact%', 'Z-Contact%']]
fg_select_data

,Name,Season,AVG,ISO,wRC+,BB%,K%,Barrel%,HardHit%,EV,maxEV,LA,GB%,FB%,SwStr%,CStr%,O-Swing%,Z-Swing%,O-Contact%,Z-Contact%
0,Aaron Judge,2024,0.322,0.379,218,0.189,0.243,0.269,0.609,96.2,117.5,18.9,0.305,0.462,0.121,0.146,0.213,0.719,0.489,0.807
1,Aaron Judge,2022,0.311,0.375,206,0.159,0.251,0.262,0.609,95.8,118.4,14.9,0.373,0.435,0.118,0.169,0.268,0.676,0.517,0.852
2,Bobby Witt Jr.,2024,0.332,0.256,168,0.080,0.150,0.143,0.481,92.7,116.9,15.1,0.370,0.444,0.098,0.138,0.354,0.748,0.700,0.884
3,Mookie Betts,2018,0.346,0.294,185,0.132,0.148,0.131,0.500,92.3,110.6,18.5,0.339,0.449,0.050,0.220,0.198,0.570,0.706,0.930
4,Mike Trout,2018,0.312,0.316,188,0.201,0.204,0.153,0.460,91.2,118.0,18.6,0.313,0.453,0.060,0.201,0.218,0.591,0.690,0.917
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1228,Jurickson Profar,2023,0.242,0.126,78,0.096,0.173,0.040,0.317,86.5,108.8,14.8,0.401,0.372,0.085,0.151,0.277,0.724,0.664,0.897
1229,Mark Trumbo,2017,0.234,0.163,79,0.070,0.247,0.063,0.375,89.6,118.5,12.5,0.433,0.406,0.138,0.154,0.344,0.708,0.597,0.808
1230,Victor Martinez,2018,0.251,0.103,73,0.063,0.096,0.042,0.304,87.8,107.6,14.1,0.394,0.361,0.064,0.148,0.338,0.698,0.800,0.914
1231,Albert Pujols,2017,0.241,0.145,77,0.058,0.146,0.048,0.391,88.7,112.2,13.5,0.435,0.381,0.088,0.180,0.339,0.644,0.676,0.905


In [8]:
numeric_cols = fg_select_data.select_dtypes(include = ['number']).columns
numeric_data = fg_select_data[numeric_cols[1:]]
numeric_data

,AVG,ISO,wRC+,BB%,K%,Barrel%,HardHit%,EV,maxEV,LA,GB%,FB%,SwStr%,CStr%,O-Swing%,Z-Swing%,O-Contact%,Z-Contact%
0,0.322,0.379,218,0.189,0.243,0.269,0.609,96.2,117.5,18.9,0.305,0.462,0.121,0.146,0.213,0.719,0.489,0.807
1,0.311,0.375,206,0.159,0.251,0.262,0.609,95.8,118.4,14.9,0.373,0.435,0.118,0.169,0.268,0.676,0.517,0.852
2,0.332,0.256,168,0.080,0.150,0.143,0.481,92.7,116.9,15.1,0.370,0.444,0.098,0.138,0.354,0.748,0.700,0.884
3,0.346,0.294,185,0.132,0.148,0.131,0.500,92.3,110.6,18.5,0.339,0.449,0.050,0.220,0.198,0.570,0.706,0.930
4,0.312,0.316,188,0.201,0.204,0.153,0.460,91.2,118.0,18.6,0.313,0.453,0.060,0.201,0.218,0.591,0.690,0.917
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1228,0.242,0.126,78,0.096,0.173,0.040,0.317,86.5,108.8,14.8,0.401,0.372,0.085,0.151,0.277,0.724,0.664,0.897
1229,0.234,0.163,79,0.070,0.247,0.063,0.375,89.6,118.5,12.5,0.433,0.406,0.138,0.154,0.344,0.708,0.597,0.808
1230,0.251,0.103,73,0.063,0.096,0.042,0.304,87.8,107.6,14.1,0.394,0.361,0.064,0.148,0.338,0.698,0.800,0.914
1231,0.241,0.145,77,0.058,0.146,0.048,0.391,88.7,112.2,13.5,0.435,0.381,0.088,0.180,0.339,0.644,0.676,0.905


In [9]:
corr_heatmap = px.imshow(numeric_data.corr(), text_auto=True)
corr_heatmap.update_layout(width=800, height=800)

In [10]:
numeric_data.describe()

,AVG,ISO,wRC+,BB%,K%,Barrel%,HardHit%,EV,maxEV,LA,GB%,FB%,SwStr%,CStr%,O-Swing%,Z-Swing%,O-Contact%,Z-Contact%
count,1233.000000,1233.000000,1233.000000,1233.000000,1233.000000,1233.000000,1233.000000,1233.000000,1233.000000,1233.000000,1233.000000,1233.000000,1233.000000,1233.000000,1233.000000,1233.000000,1233.000000,1233.000000
mean,0.266499,0.184259,112.509327,0.088322,0.198298,0.077414,0.388789,89.213544,111.466748,12.734225,0.425007,0.367054,0.101031,0.161269,0.311842,0.688252,0.654454,0.867915
std,0.027121,0.054427,22.477119,0.030972,0.053702,0.040836,0.077165,2.153833,3.059852,4.293161,0.064489,0.066306,0.031241,0.031139,0.057665,0.059316,0.081979,0.045505
min,0.168000,0.051000,46.000000,0.015000,0.043000,0.000000,0.085000,80.500000,101.900000,-4.400000,0.243000,0.150000,0.025000,0.075000,0.151000,0.487000,0.422000,0.715000
25%,0.248000,0.147000,97.000000,0.066000,0.159000,0.048000,0.344000,87.900000,109.400000,9.800000,0.380000,0.323000,0.078000,0.140000,0.271000,0.648000,0.596000,0.839000
50%,0.265000,0.180000,111.000000,0.086000,0.197000,0.074000,0.395000,89.200000,111.400000,12.900000,0.423000,0.368000,0.100000,0.161000,0.310000,0.687000,0.656000,0.871000
75%,0.284000,0.221000,126.000000,0.107000,0.235000,0.103000,0.439000,90.600000,113.400000,15.600000,0.468000,0.414000,0.122000,0.182000,0.347000,0.730000,0.709000,0.902000
max,0.354000,0.379000,218.000000,0.222000,0.372000,0.269000,0.609000,96.200000,122.200000,24.400000,0.674000,0.548000,0.217000,0.273000,0.495000,0.855000,0.914000,0.975000


## Univariate Data Analysis

In [11]:
def plot_histograms(df: pd.DataFrame) -> None:    
    fig = make_subplots(
        rows=len(df.columns),
        cols=1,
        subplot_titles=[f"Histogram for {col}" for col in df.columns]
    )

    for i, column in enumerate(df.columns):
        fig.add_trace(
            go.Histogram(x=df[column], name=column),
            row=i+1,
            col=1
        )

        fig.add_vline(x = df[column].mean(),
                    row=i+1, col=1, line_dash="dash",
                    annotation_text="Mean",
                    annotation_position="top right")

    fig.update_layout(height=400*len(df.columns), title_text="Histograms for All Columns")
    fig.show()


In [12]:
plot_histograms(numeric_data)

In [13]:
avg = numeric_data['AVG']
avg[(0.1 < avg) & (avg < 0.3)]

6       0.299
9       0.284
11      0.297
12      0.296
13      0.288
        ...  
1228    0.242
1229    0.234
1230    0.251
1231    0.241
1232    0.168
Name: AVG, Length: 1087, dtype: float64

In [14]:
#Removing outliers
def remove_outliers(col: pd.Series) -> pd.Series:
    q1 = col.quantile(0.25)
    q3 = col.quantile(0.75)

    iqr = q3 - q1
    lower = q1 - 1.5*iqr
    upper = q3 + 1.5*iqr

    return col[(lower < col) & (col < upper)] 

numeric_data_no_outliers = numeric_data.apply(remove_outliers).dropna().reset_index(drop = True)
numeric_data_no_outliers

,AVG,ISO,wRC+,BB%,K%,Barrel%,HardHit%,EV,maxEV,LA,GB%,FB%,SwStr%,CStr%,O-Swing%,Z-Swing%,O-Contact%,Z-Contact%
0,0.332,0.256,168.0,0.080,0.150,0.143,0.481,92.7,116.9,15.1,0.370,0.444,0.098,0.138,0.354,0.748,0.700,0.884
1,0.297,0.271,154.0,0.103,0.187,0.114,0.465,92.5,113.6,10.4,0.448,0.379,0.110,0.145,0.251,0.707,0.602,0.828
2,0.281,0.248,155.0,0.108,0.221,0.112,0.539,92.8,113.1,9.2,0.467,0.326,0.098,0.167,0.262,0.686,0.623,0.858
3,0.305,0.324,161.0,0.144,0.164,0.126,0.456,91.1,110.6,17.9,0.315,0.424,0.097,0.149,0.268,0.704,0.695,0.830
4,0.286,0.246,157.0,0.136,0.121,0.074,0.398,89.4,107.6,16.8,0.346,0.434,0.043,0.209,0.200,0.605,0.791,0.928
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1107,0.241,0.092,63.0,0.042,0.125,0.005,0.239,84.5,105.5,9.1,0.517,0.281,0.077,0.188,0.348,0.658,0.730,0.912
1108,0.242,0.126,78.0,0.096,0.173,0.040,0.317,86.5,108.8,14.8,0.401,0.372,0.085,0.151,0.277,0.724,0.664,0.897
1109,0.234,0.163,79.0,0.070,0.247,0.063,0.375,89.6,118.5,12.5,0.433,0.406,0.138,0.154,0.344,0.708,0.597,0.808
1110,0.251,0.103,73.0,0.063,0.096,0.042,0.304,87.8,107.6,14.1,0.394,0.361,0.064,0.148,0.338,0.698,0.800,0.914


In [15]:
plot_histograms(numeric_data_no_outliers)

In [31]:
target_column = "wRC+"

correlations = numeric_data_no_outliers.corrwith(numeric_data_no_outliers[target_column])
correlations.drop('wRC+')

AVG           0.603215
ISO           0.677111
BB%           0.439644
K%           -0.042751
Barrel%       0.497441
HardHit%      0.482582
EV            0.485510
maxEV         0.324849
LA            0.184283
GB%          -0.249883
FB%           0.198118
SwStr%       -0.028629
CStr%        -0.094448
O-Swing%     -0.211107
Z-Swing%      0.061008
O-Contact%   -0.020716
Z-Contact%   -0.072296
dtype: float64

In [16]:
numeric_data_no_outliers.to_csv('../data/fangraphs_data_simplified_cleaned', index = False)